In [20]:
import sys
sys.path.append("../../")
from taxcalc import *
import pandas as pd
import numpy as np

In [29]:
# The baseline policy

policy_xx = Policy()
behavior_xx = Behavior()
records_xx = Records("../../puf.csv")

In [30]:
# The Reform version 1

# Set 10% and 15% brackets to 12%
# Set 25% and 28% brackets to 25%
# Set 33%, 35% and 39.6% brackets to 33%
# Repeal Alternative minumum tax
# Standard deduction 12K for single/separate, 24K for married, 18K for single with child(Head of HH)
#    Should we eliminate the additional standard deduction for the aged and blind?
# Personal exemption '_II_credit' set to zero
# Child tax credit and non-child dependent credit implemented oddly
#    Dependent standard deduction set to $500, and child tax credit set to $1000
#    Should mimic non-refundable $500 deduction for non-child dependent and $1500 child tax credit
# Child tax credit phaseout for married couples increased to $150K
# Eliminated all itemized deductions except mortgage and charity
# Excluded half of interest income from final taxable interest income
# Cap gains & dividend income changed
#    Second cap gains threshold moved to same as fourth income tax bracket
#    Rates adjusted to 6%, 12.5% and 16.5%

# Didn't implement business pass-thru rate to 25%
# Didn't implement change from depreciation to expensing
# Didn't deal with eliminating net interest expense deductions for businesses
# Didn't eliminate Section 199
# Didn't implement changes to net operating losses
# Didn't implement tax reforms in Better Way health reforms plan




policy_y1 = Policy()
behavior_y1 = Behavior()
records_y1 = Records("../../puf.csv")
reform_y1 = {
    2017: {
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt3': [0.25],
        '_II_rt4': [0.25],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33],
        '_AMT_trt1': [0.0],
        '_AMT_trt2': [0.0],
        '_STD': [[12000, 24000, 12000, 18000, 24000, 12000, 500]],
        '_II_em': [0,0,0,0],
        '_CTC_c': [1500],
        '_CTC_ps': [[75000, 150000, 55000, 75000, 75000, 55000]],
        '_ID_StateLocalTax_HC': [1.0],
        '_ID_Medical_HC': [1.0],
        '_ID_Casualty_HC': [1.0],
        '_ID_Miscellaneous_HC': [1.0],
        '_ID_RealEstate_HC': [1.0],
        '_ID_BenefitSurtax_crt': [0.0],
        '_ALD_Interest_ec': [0.5],        
        '_CG_thd2': [[183250, 223050, 111525, 203150, 223050, 111525],
                     [186350, 226850, 113425, 206600, 226850, 113425],
                     [189300, 230450, 115225, 209850, 230450, 115225],
                     [190150, 231450, 115725, 210800, 231450, 115725]],
        '_CG_rt1': [0.06],
        '_CG_rt2': [0.125],
        '_CG_rt3': [0.165]
    }
}

policy_y1.implement_reform(reform_y1)

In [31]:
calc_xx = Calculator(policy_xx, records_xx, behavior_xx)
for i in range(4):
    calc_xx.increment_year()
assert calc_xx.current_year == 2017
calc_y1 = Calculator(policy_y1, records_y1, behavior_y1)
for i in range(4):
    calc_y1.increment_year()
assert calc_y1.current_year == 2017

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [32]:
print calc_xx.current_year
print calc_y1.current_year

2017
2017


In [33]:
calc_xx.calc_all()
calc_y1.calc_all()

In [34]:
calc_xx_combined = (calc_xx.records._combined * calc_xx.records.s006)
calc_y1_combined = (calc_y1.records._combined * calc_y1.records.s006)

diff_static_y1_xx = (calc_y1_combined - calc_xx_combined).sum() / 10**9
print "Static net revenue change: " + str(diff_static_y1_xx) + " billion dollars"

Static net revenue change: -13.7289520135 billion dollars


In [37]:
# Behavioral factors
# Set income effect to 0
# Set substitution effect to 0.3
behavior_xx.update_behavior({2017: {'_BE_inc': [0], '_BE_sub': [0.3], '_BE_cg': [0.2]}})
behavior_y1.update_behavior({2017: {'_BE_inc': [0], '_BE_sub': [0.3], '_BE_cg': [0.2]}})

#calc_xx = Behavior(calc_xx, calc_xx)
calc_y1_beh = Behavior.response(calc_xx, calc_y1)
calc_y1_beh.calc_all()



In [39]:
calc_y1_beh_combined = (calc_y1_beh.records._combined * calc_y1_beh.records.s006)
diff_Beh_y1_xx = (calc_y1_beh_combined - calc_xx_combined).sum() / 10**9
print "With behavioral effects, net revenue change: " + str(diff_Beh_y1_xx) + " billion dollars"

With behavioral effects, net revenue change: -13.7289520135 billion dollars


In [40]:
# Individual reforms and their effects

# AMT repeal
policy_amt = Policy()
behavior_amt = Behavior()
records_amt = Records("../../puf.csv")
reform_amt = {
    2017: {
        '_AMT_trt1': [0.0],
        '_AMT_trt2': [0.0]
    }
}
policy_amt.implement_reform(reform_amt)

# Change in income tax rates
policy_IncTax = Policy()
behavior_IncTax = Behavior()
records_IncTax = Records("../../puf.csv")
reform_IncTax = {
    2017: {
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt3': [0.25],
        '_II_rt4': [0.25],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33]
    }
}
policy_IncTax.implement_reform(reform_IncTax)

# Deduction reforms and child tax credit
policy_ded = Policy()
behavior_ded = Behavior()
records_ded = Records("../../puf.csv")
reform_ded = {
    2017: {
        '_STD': [[12000, 24000, 12000, 18000, 24000, 12000, 500]],
        '_II_em': [0,0,0,0],
        '_CTC_c': [1500],
        '_CTC_ps': [[75000, 150000, 55000, 75000, 75000, 55000]],
        '_ID_StateLocalTax_HC': [1.0],
        '_ID_Medical_HC': [1.0],
        '_ID_Casualty_HC': [1.0],
        '_ID_Miscellaneous_HC': [1.0],
        '_ID_RealEstate_HC': [1.0],
        '_ID_BenefitSurtax_crt': [0.0]
    }
}
policy_ded.implement_reform(reform_ded)

# Investment income (interest, cap gains and dividends) reforms
policy_InvTax = Policy()
behavior_InvTax = Behavior()
records_InvTax = Records("../../puf.csv")
reform_InvTax = {
    2017: {
        '_CG_thd2': [[183250, 223050, 111525, 203150, 223050, 111525],
                     [186350, 226850, 113425, 206600, 226850, 113425],
                     [189300, 230450, 115225, 209850, 230450, 115225],
                     [190150, 231450, 115725, 210800, 231450, 115725]],
        '_CG_rt1': [0.06],
        '_CG_rt2': [0.125],
        '_CG_rt3': [0.165]
    }
}
policy_InvTax.implement_reform(reform_InvTax)

In [42]:
calc_amt = Calculator(policy_amt, records_amt, behavior_amt)
for i in range(4):
    calc_amt.increment_year()
assert calc_amt.current_year == 2017

calc_IncTax = Calculator(policy_IncTax, records_IncTax, behavior_IncTax)
for i in range(4):
    calc_IncTax.increment_year()
assert calc_IncTax.current_year == 2017

calc_ded = Calculator(policy_ded, records_ded, behavior_ded)
for i in range(4):
    calc_ded.increment_year()
assert calc_ded.current_year == 2017

calc_InvTax = Calculator(policy_InvTax, records_InvTax, behavior_InvTax)
for i in range(4):
    calc_InvTax.increment_year()
assert calc_InvTax.current_year == 2017

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [43]:
print calc_amt.current_year
print calc_IncTax.current_year
print calc_ded.current_year
print calc_InvTax.current_year

2017
2017
2017
2017


In [44]:
calc_amt.calc_all()
calc_IncTax.calc_all()
calc_ded.calc_all()
calc_InvTax.calc_all()

In [45]:
calc_amt_combined = (calc_amt.records._combined * calc_amt.records.s006)
calc_IncTax_combined = (calc_IncTax.records._combined * calc_IncTax.records.s006)
calc_ded_combined = (calc_ded.records._combined * calc_ded.records.s006)
calc_InvTax_combined = (calc_InvTax.records._combined * calc_InvTax.records.s006)

diff_static_amt_xx = (calc_amt_combined - calc_xx_combined).sum() / 10**9
diff_static_IncTax_xx = (calc_IncTax_combined - calc_xx_combined).sum() / 10**9
diff_static_ded_xx = (calc_ded_combined - calc_xx_combined).sum() / 10**9
diff_static_InvTax_xx = (calc_InvTax_combined - calc_xx_combined).sum() / 10**9

In [46]:
print "Static net revenue change with AMT repeal: " + str(diff_static_amt_xx) + " billion dollars"
print "Static net revenue change with income tax reform: " + str(diff_static_IncTax_xx) + " billion dollars"
print "Static net revenue change with deduction reform: " + str(diff_static_ded_xx) + " billion dollars"
print "Static net revenue change with investment tax reform: " + str(diff_static_InvTax_xx) + " billion dollars"

Static net revenue change with AMT repeal: -41.8800806925 billion dollars
Static net revenue change with income tax reform: -98.4209110176 billion dollars
Static net revenue change with deduction reform: 162.483433836 billion dollars
Static net revenue change with investment tax reform: -2.61514808691 billion dollars
